# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [107]:
# Dependencies and Setup

import pandas as pd
import numpy as np
import os


# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [108]:
# Calculate the total number of unique schools
school_count_df =school_data_complete_df["school_name"].nunique()
school_count_df


15

In [109]:
# Calculate the total number of students
student_count_df = school_data_complete_df["student_name"].count()
student_count_df

39170

In [110]:
# Calculate the total budget
total_budget_df =school_data_complete_df["budget"].sum()
total_budget_df

82932329558

In [111]:
# Calculate the average (mean) math score
average_math_score_df =school_data_complete_df["math_score"].mean()
average_math_score_df

78.98537145774827

In [112]:
# Calculate the average (mean) reading score
average_reading_score_df =school_data_complete_df["reading_score"].mean()
average_reading_score_df

81.87784018381414

In [113]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count_df = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_math_percentage_df = passing_math_count_df / float(student_count_df) * 100
passing_math_percentage_df

74.9808526933878

In [114]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count_df = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage_df = passing_reading_count_df / float(student_count_df) * 100
passing_reading_percentage_df

85.80546336482001

In [115]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count_df = school_data_complete_df[
    (school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate_df = passing_math_reading_count_df /  float(student_count_df) * 100
overall_passing_rate_df

65.17232575950983

In [116]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary_df = pd.DataFrame({
    "Total Schools": school_count_df,
    "Total Students": f"{student_count_df:,}",
    "Total Budget": f"${total_budget_df:,.2f}",
    "Average Math Score": f"{average_math_score_df:.2f}",
    "Average Reading Score": f"{average_reading_score_df:.2f}",
    "% Passing Math": f"{passing_math_percentage_df:.2f}",
    "% Passing Reading": f"{passing_reading_percentage_df:.2f}",
    "% Overall Passing": f"{overall_passing_rate_df:.2f}"
}, index=[0])

district_summary_df



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$82,932,329,558.00",78.99,81.88,74.98,85.81,65.17


## School Summary

In [117]:
# Use the code provided to select the school type
school_types_df = school_data.set_index(["school_name"])["type"]
school_types_df

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [118]:
# Calculate the total student count
per_school_counts_df =school_data_complete_df.set_index(["student_name"])["school_name"].value_counts()
per_school_counts_df

Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: school_name, dtype: int64

In [119]:
# Calculate the total school budget and per capita spending
per_school_budget_df = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
per_school_capita_df = per_school_budget_df / per_school_counts_df
per_school_capita_df
                        


Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [120]:
# Calculate the average test scores
per_school_math_df = school_data_complete_df.groupby(["school_name"])["math_score"].mean()
per_school_math_df
#per_school_reading = 


school_name
Bailey High School       77.048432
Cabrera High School      83.061895
Figueroa High School     76.711767
Ford High School         77.102592
Griffin High School      83.351499
Hernandez High School    77.289752
Holden High School       83.803279
Huang High School        76.629414
Johnson High School      77.072464
Pena High School         83.839917
Rodriguez High School    76.842711
Shelton High School      83.359455
Thomas High School       83.418349
Wilson High School       83.274201
Wright High School       83.682222
Name: math_score, dtype: float64

In [121]:
per_school_reading_df=school_data_complete_df.groupby(["school_name"])["reading_score"].mean()
per_school_reading_df

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [122]:
# Calculate the number of schools with math scores of 70 or higher 
school_passing_math_df = school_data_complete_df[(school_data_complete_df["math_score"]>=70)]
school_passing_math_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [123]:
# Calculate the number of schools with reading scores of 70 or higher 

school_passing_reading_df = school_data_complete_df[(school_data_complete_df["reading_score"]>=70)]
school_passing_reading_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [124]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading_df = school_data_complete_df[
    (school_data_complete_df["reading_score"] >= 70) & (school_data_complete_df["math_score"] >= 70)
]
passing_math_and_reading_df


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [125]:
# Use the provided code to calculate the passing rates
per_school_passing_math_df = school_passing_math_df.groupby(["school_name"]).count()["student_name"] / per_school_counts_df * 100
per_school_passing_reading_df = school_passing_reading_df.groupby(["school_name"]).count()["student_name"] / per_school_counts_df * 100
overall_passing_rate_df = passing_math_and_reading_df.groupby(["school_name"]).count()["student_name"] / per_school_counts_df * 100
overall_passing_rate_df

Bailey High School       54.642283
Cabrera High School      91.334769
Figueroa High School     53.204476
Ford High School         54.289887
Griffin High School      90.599455
Hernandez High School    53.527508
Holden High School       89.227166
Huang High School        53.513884
Johnson High School      53.539172
Pena High School         90.540541
Rodriguez High School    52.988247
Shelton High School      89.892107
Thomas High School       90.948012
Wilson High School       90.582567
Wright High School       90.333333
dtype: float64

In [126]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary_df=pd.DataFrame({
    "Schools Passing Math": per_school_passing_math_df,
    "Schools Passing Reading": per_school_passing_reading_df,
    "Passing Both Math and Reading":overall_passing_rate_df})



# Formatting
#per_school_summary_df["per_school_capita_df"] = per_school_summary_df["per_school_capita_df"].map("${:,.2f}".format)
per_school_summary_df.style.format({"% Passing Math": "{:2f}", 
                                    "% Passing Reading": "{:2f}",
                                    "Passing Both Math and Reading":"{:2f}"})

# Display the DataFrame
per_school_summary_df

,Schools Passing Math,Schools Passing Reading,Passing Both Math and Reading
Bailey High School,66.680064,81.933280,54.642283
Cabrera High School,94.133477,97.039828,91.334769
Figueroa High School,65.988471,80.739234,53.204476
Ford High School,68.309602,79.299014,54.289887
Griffin High School,93.392371,97.138965,90.599455
Hernandez High School,66.752967,80.862999,53.527508
Holden High School,92.505855,96.252927,89.227166
Huang High School,65.683922,81.316421,53.513884
Johnson High School,66.057551,81.222432,53.539172
Pena High School,94.594595,95.945946,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [127]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
school_performance_high=per_school_summary_df.nlargest(5, "Passing Both Math and Reading")
school_performance_high

,Schools Passing Math,Schools Passing Reading,Passing Both Math and Reading
Cabrera High School,94.133477,97.039828,91.334769
Thomas High School,93.272171,97.308869,90.948012
Griffin High School,93.392371,97.138965,90.599455
Wilson High School,93.867718,96.539641,90.582567
Pena High School,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [128]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
school_performance_low=per_school_summary_df.nsmallest(5, "Passing Both Math and Reading")
school_performance_low

,Schools Passing Math,Schools Passing Reading,Passing Both Math and Reading
Rodriguez High School,66.366592,80.220055,52.988247
Figueroa High School,65.988471,80.739234,53.204476
Huang High School,65.683922,81.316421,53.513884
Hernandez High School,66.752967,80.862999,53.527508
Johnson High School,66.057551,81.222432,53.539172


## Math Scores by Grade

In [129]:
# Use the code provided to separate the data by grade
ninth_graders_df = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders_df = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders_df = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders_df = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]


In [130]:
# Group by "school_name" and take the mean of each.
ninth_math = student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["math_score"].mean()
tenth_math = student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["math_score"].mean()
eleventh_math = student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["math_score"].mean()
twelfth_math = student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["math_score"].mean()

In [131]:
# Use the code to select only the `math_score`.
math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math})

In [132]:
# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade=math_scores


# Minor data wrangling
math_scores_by_grade.index.name = None

                                                             
# Display the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [133]:
# Use the code provided to separate the data by grade
ninth_df = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_df = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_df = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_df = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

# Group by "school_name" 
ninth_reading = student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["reading_score"].mean()
tenth_reading = student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
eleventh_reading = student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
twelfth_reading = student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

reading_scores=pd.DataFrame({
    "9th":ninth_reading,
    "10th":tenth_reading,
    "11th":eleventh_reading,
    "12th":twelfth_reading})

# Use the code to select only the `reading_score`.

reading_scores_by_grade=reading_scores


# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [134]:
# Establish the bins 
bins = [0, 585, 630, 645, 680]
group_spending = ["<$585", "$585-630", "$630-645", "$645-680"]


In [135]:
# Create a copy of the school summary since it has the "Per Student Budget" 
scores_by_school_spending_df = per_school_summary_df.copy()

In [136]:
# Use `pd.cut` to categorize spending based on the bins.
school_data_complete_df["spending_bins"] = pd.cut(school_data_complete_df["budget"]/school_data_complete_df["size"], bins, labels = group_spending)
school_data_complete_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,spending_bins
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,$645-680
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,$645-680
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,$645-680
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,$645-680
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,$645-680
...,...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130,$630-645
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130,$630-645
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130,$630-645
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130,$630-645


In [137]:
#  Calculate averages for the desired columns. 
ave_math_spen_score=school_data_complete_df["budget"].mean() / per_school_summary_df["Schools Passing Math"]
ave_math_spen_score


Bailey High School       31752.233766
Cabrera High School      22491.902564
Figueroa High School     32085.013761
Ford High School         30994.778569
Griffin High School      22670.384920
Hernandez High School    31717.556517
Holden High School       22887.643101
Huang High School        32233.778530
Johnson High School      32051.460575
Pena High School         22382.261889
Rodriguez High School    31902.210689
Shelton High School      22555.725242
Thomas High School       22699.600117
Wilson High School       22555.581796
Wright High School       22684.724887
Name: Schools Passing Math, dtype: float64

In [138]:
ave_read_spen_score=school_data_complete_df["budget"].mean() / per_school_summary_df["Schools Passing Reading"]
ave_read_spen_score

Bailey High School       25841.037929
Cabrera High School      21818.268211
Figueroa High School     26223.198984
Ford High School         26699.461649
Griffin High School      21796.001210
Hernandez High School    26183.062930
Holden High School       21996.639964
Huang High School        26037.065625
Johnson High School      26067.195115
Pena High School         22067.018764
Rodriguez High School    26392.913706
Shelton High School      22088.041365
Thomas High School       21757.944801
Wilson High School       21931.312064
Wright High School       21915.087873
Name: Schools Passing Reading, dtype: float64

In [139]:
spending_math_scores = school_data_complete_df.groupby(["spending_bins"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (student_count_df)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (student_count_df)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (student_count_df)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (student_count_df)"]).mean()["% Overall Passing"]]

SyntaxError: unmatched ']' (3966608958.py, line 5)

In [140]:
# Assemble into DataFrame
spending_summary = pd.DataFrame(
    [{"Ave Spending Math":ave_math_spen_score,
     "Ave Spending Reading": ave_read_spen_score}])

# Display results
spending_summary

,Ave Spending Math,Ave Spending Reading
0,Bailey High School 31752.233766 Cabrera ...,Bailey High School 25841.037929 Cabrera ...


## Scores by School Size

In [141]:
# Establish the bins.
bins_size = [0, 1000, 2000, 5000]
group_size = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

In [142]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
scores_by_school_spending_df = per_school_summary_df.copy()

school_data_complete_df["size_bins"] = pd.cut(student_count_df["student_name"].nunique()/school_data_complete_df["size"], bins_size, labels = group_size)

school_data_complete_df

#per_school_summary["School Size"] = 

IndexError: invalid index to scalar variable.

In [143]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

NameError: name 'per_school_summary' is not defined

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`


# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = 
type_reading_scores = 
type_passing_math = 
type_passing_reading = 
type_overall_passing = 

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

In [144]:
# Assemble the new data by type into a DataFrame called `type_summary`


# Display results
type_summary

NameError: name 'type_summary' is not defined